In [0]:
courses = [
    {   
        'course_id': 1,
        'course_name': "Python Bootcamp",
        'suitable_for': 'Beginner',
        'enrollment': 1100093,
        'stars': 4.6,
        'number_of_ratings': 218066
    },
    {   
        'course_id': 2,
        'course_name': "angular - the complete guide",
        'suitable_for': 'Intermediate',
        'enrollment': 34567,
        'stars': 4.5,
        'number_of_ratings': 347912
    },
    {
        'course_id': 3,
        'course_name': "Java In-Depth",
        'suitable_for': 'Adavanced',
        'enrollment': 2345321,
        'stars': 4.8,
        'number_of_ratings': 23789
    },
    {
        'course_id': 4,
        'course_name': "C++ Beginner guide",
        'suitable_for': 'Beginner',
        'enrollment': 32145,
        'stars': 4.2,
        'number_of_ratings': 5678
    },
    {
        'course_id': 5,
        'course_name': "Data Science Practical approach",
        'suitable_for': 'Intermediate',
        'enrollment': 67897,
        'stars': 4.6,
        'number_of_ratings': 267576
    }
]

In [0]:

from pyspark.sql.functions import *
from pyspark.sql import Row

In [0]:
course_df = spark.createDataFrame([Row(**course) for course in courses])

In [0]:
import getpass
username = getpass.getuser()

In [0]:
help(course_df.write.mode)

Help on method mode in module pyspark.sql.readwriter:

mode(saveMode: Optional[str]) -> 'DataFrameWriter' method of pyspark.sql.readwriter.DataFrameWriter instance
    Specifies the behavior when data or table already exists.
    
    Options include:
    
    * `append`: Append contents of this :class:`DataFrame` to existing data.
    * `overwrite`: Overwrite existing data.
    * `error` or `errorifexists`: Throw an exception if data already exists.
    * `ignore`: Silently ignore this operation if data already exists.
    
    .. versionadded:: 1.4.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Examples
    --------
    Raise an error when writing to an existing path.
    
    >>> import tempfile
    >>> with tempfile.TemporaryDirectory() as d:
    ...     spark.createDataFrame(
    ...         [{"age": 80, "name": "Xinrong Meng"}]
    ...     ).write.mode("error").format("parquet").save(d) # doctest: +SKIP
    Traceback (most recent call last):
       

Different ways mode can be specified while writing data frame into files. *file_format* can be any valid out of box format such as text, csv, json, parquet. orc

- course_df.write.mode(saveMode).file_format(path_to_folder)
- course_df.write.file_format(path_to_folder,mode=saveMode)
- course_df.write.mode(saveMode).format('file_format').save(path_to_folder)
- course_df.write.format('file_format').save(path_to_folder, mode=saveMode)

- Understand default behavior
  - Fails if folder exists
  - Creates folder and then adds files to it

In [0]:
dbutils.fs.rm(f'/user/{username}/course', recurse=True)

True

In [0]:
course_df.\
    coalesce(1).\
    write.\
    parquet(
        f'/user/{username}/course'
    )

In [0]:
dbutils.fs.ls(f'/user/{username}/course')

[FileInfo(path='dbfs:/user/root/course/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1698123636000),
 FileInfo(path='dbfs:/user/root/course/_committed_6160619855608807065', name='_committed_6160619855608807065', size=123, modificationTime=1698123636000),
 FileInfo(path='dbfs:/user/root/course/_started_6160619855608807065', name='_started_6160619855608807065', size=0, modificationTime=1698123635000),
 FileInfo(path='dbfs:/user/root/course/part-00000-tid-6160619855608807065-c6c000b0-c19f-4aa7-9038-b055c81d5507-25-1.c000.snappy.parquet', name='part-00000-tid-6160619855608807065-c6c000b0-c19f-4aa7-9038-b055c81d5507-25-1.c000.snappy.parquet', size=2031, modificationTime=1698123636000)]

In [0]:
# fails as mode is error or errorIfExists by default
course_df.\
    coalesce(1).\
    write.\
    parquet(
        f'/user/{username}/course'
    )

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2792568061432263>, line 5
      1 # fails as mode is error or errorIfExists by default
      2 course_df.\
      3     coalesce(1).\
      4     write.\
----> 5     parquet(
      6         f'/user/{username}/course'
      7     )

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1721, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1719     self.partitionBy(partitionBy)
   1720 self._set_opts(compression=compression)
-> 172

In [0]:
spark.read.parquet(f'/user/{username}/course').count()

5

In [0]:
course_df.\
    coalesce(1).\
    write.\
    mode('overwrite').\
    parquet(f'/user/{username}/course')

In [0]:
course_df.\
    coalesce(1).\
    write.\
    parquet(f'/user/{username}/course', mode = 'overwrite')

In [0]:
course_df.\
    coalesce(1).\
    write.\
    format('parquet').\
    save(f'/user/{username}/course',mode = 'overwrite')

In [0]:
course_df.\
    coalesce(1).\
    write.\
    mode('append').\
    parquet(f'/user/{username}/course')

In [0]:
dbutils.fs.ls(f'/user/{username}/course')

[FileInfo(path='dbfs:/user/root/course/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1698124256000),
 FileInfo(path='dbfs:/user/root/course/_committed_2775427910264764123', name='_committed_2775427910264764123', size=232, modificationTime=1698123976000),
 FileInfo(path='dbfs:/user/root/course/_committed_4562339149116089518', name='_committed_4562339149116089518', size=221, modificationTime=1698124196000),
 FileInfo(path='dbfs:/user/root/course/_committed_6116636353908954032', name='_committed_6116636353908954032', size=221, modificationTime=1698124112000),
 FileInfo(path='dbfs:/user/root/course/_committed_6160619855608807065', name='_committed_6160619855608807065', size=123, modificationTime=1698123636000),
 FileInfo(path='dbfs:/user/root/course/_committed_7233770919239630562', name='_committed_7233770919239630562', size=123, modificationTime=1698124255000),
 FileInfo(path='dbfs:/user/root/course/_started_2775427910264764123', name='_started_2775427910264764123', size=0, modific

In [0]:
spark.read.parquet(f'/user/{username}/course').count()

10